 **install textblob**

In [75]:
!pip3 install textblob
!pip3 install googletrans
!pip3 install pygal
!pip3 install pandas
!pip3 install geopandas
!pip3 install matplotlib
!pip3 install numpy
!pip3 install plotly_express

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**imports**

In [3]:
# -*- coding: utf-8 -*-
from textblob import TextBlob
from googletrans import Translator
from unidecode import unidecode
from IPython.display import SVG, HTML, display
from random import randint
import statistics
import pygal
import datetime
import calendar
import json
import time
import csv
import pandas as pd

translator = Translator()

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))

<font size="4">**Data Process**</font>

<font size="3">**Tranlate file and salve in other file**</font>

In [ ]:
# it need a json Key available on google API console and then RUN
# export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/my-key.json"

def googleTranslator(text, target):
    target = "en"
    translate_client = translate.Client()
    result = translate_client.translate(text, target_language=target)
    return result


def load():

    dataset = open('data/dataset-v2.txt', 'r', encoding='utf-8')
    data_enUS = open('data/dataset-us-v4.txt', 'a', encoding='utf-8')
    data_error = open('data/dataset-error.json', 'a', encoding='utf-8')

    i = 0; # create an index
    target = "en-US" # target that will be translated
    #emoji pattern
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "]+", flags=re.UNICODE)
    aux= []
    canITranslate = False # manual fix
    latestId = '301420767' # we use only manual fix

    for line in dataset:
        try:
            #increment index - just for fun :)
            i  = i + 1
            # parse to json format
            data_json = json.loads(str(line))
            aux = data_json
            #remove emojis from review
            review_str = emoji_pattern.sub(r'', str(data_json['reviewBody']))
            #remove emojis from title
            title_str = emoji_pattern.sub(r'', str(data_json['title']))

            if(canITranslate):

                #translatting
                review_str = googleTranslator(review_str, target)
                title_str = googleTranslator(title_str, target)
                #assingement
                data_json['title'] = title_str['translatedText']
                data_json['reviewBody'] = review_str['translatedText']

            if( i > 95000): #manual_fix
                #write
                data_enUS.write(json.dumps(data_json) + "\n")
                print('p4',i)

            if( data_json['reviewId'] == latestId):
                canITranslate = True
                print('I can translate now!!')

            if( data_json['reviewId'] != latestId and canITranslate == False):
                print('looking for')

        except KeyError:
            print("there is a key error")
            data_error.write(json.dumps(aux) + "\n")

    #close files streams
    data_enUS.close()
    dataset.close()
    data_error.close()
    print('data translate finish!!')

load()

<font size="3">**Calculate subjectivity and polarity from sentences**</font>

In [127]:
import statistics

data_us = open('data/dataset-full.txt', 'r', encoding='utf-8')
data_result = open('data/dataset-resultv2.txt', 'a', encoding='utf-8')

evaluations = {}
cities = {}
mobile =0   
        
for sentence in data_us:    
    try:
        
        data = sentence.replace('\\', '').replace('\n', '')
        data = json.loads(data)
        calc = TextBlob(data['reviewBody'])
        
        data['subjectivity'] = calc.sentiment.subjectivity
        data['polarity'] = calc.sentiment.polarity
        
        #how many mobile users ?
        if(str(data['mobileDevice']) == 'True'):
            mobile += 1
        
        if data['restaurantId'] in evaluations:
            
            evaluations[data['restaurantId']]['evaluations'].append(data['polarity'])
            evaluations[data['restaurantId']]['total']= evaluations[data['restaurantId']]['total'] + 1
            evaluations[data['restaurantId']]['stars'].append(int(data['ratingValue']))
            
            if(data['userLocation'] not in evaluations[data['restaurantId']]['userLocations']):
                evaluations[data['restaurantId']]['userLocations'].append(data['userLocation'])

        else:
            evaluations[data['restaurantId']] = { 'evaluations': [], 'media': 0, 'userLocations':[], 'media_star': 0, 'total': 0,  'rest_name': '', 'stars': [] }
            evaluations[data['restaurantId']]['evaluations'].append(data['polarity'])
            evaluations[data['restaurantId']]['total'] = evaluations[data['restaurantId']]['total'] + 1
            evaluations[data['restaurantId']]['rest_name'] = data['itemReviewed']['name']
            evaluations[data['restaurantId']]['stars'].append(int(data['ratingValue']))
            evaluations[data['restaurantId']]['userLocations'].append(data['userLocation'])
                

        data_result.write(json.dumps(data) + "\n")

    except Exception as exc:
        print(exc)
    

data_us.close()
data_result.close()


Expecting ',' delimiter: line 1 column 640 (char 639)
Expecting ',' delimiter: line 1 column 13 (char 12)
'reviewBody'
'reviewBody'
Expecting ',' delimiter: line 1 column 24 (char 23)
Expecting ',' delimiter: line 1 column 13 (char 12)
Expecting ',' delimiter: line 1 column 13 (char 12)
Expecting ',' delimiter: line 1 column 1046 (char 1045)
Expecting ',' delimiter: line 1 column 13 (char 12)
Expecting ',' delimiter: line 1 column 14 (char 13)
Expecting ',' delimiter: line 1 column 14 (char 13)
Expecting ',' delimiter: line 1 column 525 (char 524)
Expecting ',' delimiter: line 1 column 418 (char 417)
Expecting ',' delimiter: line 1 column 13 (char 12)
Expecting ',' delimiter: line 1 column 22 (char 21)
Expecting ',' delimiter: line 1 column 400 (char 399)
Expecting ',' delimiter: line 1 column 164 (char 163)
Expecting ',' delimiter: line 1 column 13 (char 12)
Expecting ',' delimiter: line 1 column 21 (char 20)
Expecting ',' delimiter: line 1 column 420 (char 419)
Expecting ',' delimite

<font size="4">**Geral data**</font>

<font size="3">**Sentence Distribution**</font>

In [117]:
dataset = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')
data_star = {}

def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))
        
for sentence in dataset:
    try:
        data = json.loads(sentence)
        tam = len(data['reviewBody'].split())
        
        if ( data['ratingValue'] in data_star):
            data_star[data['ratingValue']]['words'].append(tam) 
        else:
            data_star[data['ratingValue']] = {'words': []}
            data_star[data['ratingValue']]['words'].append(tam) 
            
    except Exception as exc:
        print(exc)

one   = statistics.median(data_star['1']['words'])
two   = statistics.median(data_star['2']['words'])
three = statistics.median(data_star['3']['words'])
four  = statistics.median(data_star['4']['words'])
five  = statistics.median(data_star['5']['words'])

line_chart_pol = pygal.HorizontalBar()
line_chart_pol.title = 'Words Average by Stars'
line_chart_pol.x_labels = sorted(data_star.keys())

line_chart_pol.add('Words', [one, two, three, four, five])

galplot(line_chart_pol)

dataset.close()

<font size="3">**Daily comments**</font>

In [5]:
dataset = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')

def find_day(date):
    return datetime.datetime.strptime(date, '%d %m %Y').weekday()
        
sunday = 0
sunday_pos = 0
sunday_neg = 0
sunday_neu = 0
monday = 0
monday_pos = 0
monday_neg = 0
monday_neu = 0
tuesday = 0
tuesday_pos = 0
tuesday_neg = 0
tuesday_neu = 0
wednesday = 0
wednesday_pos = 0
wednesday_neg = 0
wednesday_neu = 0
thursday = 0
thursday_pos = 0
thursday_neg = 0
thursday_neu = 0
friday = 0
friday_pos = 0
friday_neg = 0
friday_neu = 0
saturday = 0
saturday_pos = 0
saturday_neg = 0
saturday_neu = 0


for sentence in dataset:
    try:
        data = json.loads(sentence)
        date = data['publicationDate'].replace('/', ' ')
        day = find_day(date)

        if(day == 0):
            monday += 1
            if(data['polarity'] > 0):
                monday_pos += 1
            elif(data['polarity'] < 0):
                monday_neg += 1
            else:
                monday_neu += 1
        if(day == 1):
            sunday += 1
            if(data['polarity'] > 0):
                sunday_pos += 1
            elif(data['polarity'] < 0):
                sunday_neg += 1
            else:
                sunday_neu += 1
        if(day == 2):
            tuesday += 1
            if(data['polarity'] > 0):
                tuesday_pos += 1
            elif(data['polarity'] < 0):
                tuesday_neg += 1
            else:
                tuesday_neu += 1
        if(day == 3):
            wednesday += 1
            if(data['polarity'] > 0):
                wednesday_pos += 1
            elif(data['polarity'] < 0):
                wednesday_neg += 1
            else:
                wednesday_neu += 1
        if(day == 4):
            thursday += 1
            if(data['polarity'] > 0):
                thursday_pos += 1
            elif(data['polarity'] < 0):
                thursday_neg += 1
            else:
                thursday_neu += 1
        if(day == 5):
            friday += 1
            if(data['polarity'] > 0):
                friday_pos += 1
            elif(data['polarity'] < 0):
                friday_neg += 1
            else:
                friday_neu += 1
        if(day == 6):
            saturday += 1
            if(data['polarity'] > 0):
                saturday_pos += 1
            elif(data['polarity'] < 0):
                saturday_neg += 1
            else:
                saturday_neu += 1
    except Exception as exc:
        print(exc)

line_chart_pol = pygal.HorizontalBar()
line_chart_pol.title = 'Distribution of daily comments by polarity'
line_chart_pol.x_labels = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
line_chart_pol.add('Positive', [sunday_pos, monday_pos, tuesday_pos, wednesday_pos, thursday_pos,
                          friday_pos, saturday_pos])
line_chart_pol.add('Negative', [sunday_neg, monday_neg, tuesday_neg, wednesday_neg, thursday_neg,
                          friday_neg, saturday_neg])
line_chart_pol.add('Neutral', [sunday_neu, monday_neu, tuesday_neu, wednesday_neu, thursday_neu,
                          friday_neu, saturday_neu])
galplot(line_chart_pol)

<font size="3">**Most Evaluated**</font>

In [128]:
def galplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))

arr = []

for i in evaluations:
    evaluations[i]['media'] = statistics.median(evaluations[i]['evaluations'])
    evaluations[i]['media_star'] = statistics.median(evaluations[i]['stars'])
    arr.append(evaluations[i])


dd = sorted(arr, key = lambda i: i['total'], reverse=True)[:10]
mm = sorted(arr, key = lambda i: i['media_star'], reverse=True)

line_total_rank_chart = pygal.HorizontalBar()
line_total_rank_chart.title = 'Most Revieweds'

for d in dd:
    line_total_rank_chart.add(d['rest_name'], d['total'])

galplot(line_total_rank_chart)


restaurant_rank_chart = pygal.HorizontalBar()
restaurant_rank_chart.title = 'Start Ranking'
one_star = True
two_star = True
three_star = True
four_star = True
five_star = True

for rest in mm:
    if(rest['media_star'] == 1 and one_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        one_star = False
    if(rest['media_star'] == 2 and two_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        two_star = False
    if(rest['media_star'] == 3 and three_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        three_star = False
    if(rest['media_star'] == 4 and four_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        four_star = False
    if(rest['media_star'] == 5 and five_star):
        restaurant_rank_chart.add(rest['rest_name'], rest['media_star'])
        five_star = False

galplot(restaurant_rank_chart)

<font size="3">**Polarity and subjectivity proportion**</font>

In [42]:
import pygal
from IPython.display import SVG, HTML, display

data_result = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')
        
pol_positive = 0
pol_negative = 0
pol_neutral = 0

sub_objective = 0
sub_subjective = 0

for result in data_result:
    data = json.loads(result)
    if(data['polarity'] > 0):
        pol_positive+=1
    elif(data['polarity'] < 0):
        pol_negative+=1
    else:
        pol_neutral+=1
    
    if(data['subjectivity'] < 0.5):
        sub_objective+=1
    else:
        sub_subjective+=1
    
pie_pol_chart = pygal.Pie(inner_radius=.4)
pie_pol_chart.title = 'Polarity proportion from TripAdvisor dataset'
pie_pol_chart.add('Positive', pol_positive)
pie_pol_chart.add('Negative', pol_negative)
pie_pol_chart.add('Neutral', pol_neutral)
galplot(pie_pol_chart)

pie_sub_chart = pygal.Pie(inner_radius=.4)
pie_sub_chart.title = 'Subjectivity proportion from TripAdvisor dataset'
pie_sub_chart.add('Objective', sub_objective)
pie_sub_chart.add('Subjective', sub_subjective)
galplot(pie_sub_chart)

<font size="4">**Method Evaluation**</font>

<font size="3">**Creates a random dataset**</font>

For the random dataset.csv, we thought it would be better if we did it using a script, so the data would be even more random.

In [2]:
from random import randint
import json
import csv

dataset = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')
size_dataset = 0

# Reading from file
content = dataset.read()
list_lines = content.split("\n")

# Count lines
for i in list_lines:
    if i:
        size_dataset += 1

index_memo = []

def get_especific_line(index, dataset):
    dataset = open('data/dataset-resultv2.txt', 'r', encoding='utf-8')
    for i, sentence in enumerate(dataset):
        if i == index:
            return json.loads(sentence)

def get_random_value():
    index = randint(0, size_dataset)
    
    if(index in index_memo):
        while index in index_memo:
            index = randint(0, size_dataset)

    return index

with open('data/dataset-random-200-new.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["comment", "subjectivity", "polarity", "sub-check", "pol-check"])

    for x in range(0, 199):
        index = get_random_value()    
        line = get_especific_line(index, dataset)
        print(line)
        writer.writerow([line['reviewBody'], line['subjectivity'], line['polarity']])

{'title': 'sem grau00e7a', 'reviewId': '263606766', 'restaurantId': '1180379', 'author': 'Amanda M', 'reviewBody': 'The buffet has many options, but nothing is tasty. The decor is modern, the service is reasonable and the price is good, if you buy a coupon on collective shopping sites.', 'language': 'pt', 'mobileDevice': 'False', 'userLocation': 'Rio de Janeiro, RJ', 'publicationDate': '04/04/2015', 'visitDate': '09/2014', 'ratingValue': '3', 'itemReviewed': {'name': 'Frontera', 'address': {'streetAddress': 'Avenida Armando Lombardi 295', 'addressLocality': 'Rio de Janeiro', 'addressRegion': 'Estado do Rio de Janeiro', 'postalCode': '22640-020', 'country': 'Brasil'}, 'website': 'http://www.frontera.com.br', 'email': 'None', 'phoneNumber': '+55 (21) 3154-8400', 'averageRating': '3.5'}, 'subjectivity': 1, 'polarity': 1}
{'title': 'Excelente', 'reviewId': '537009724', 'restaurantId': '1015278', 'author': 'louysecavalcante', 'reviewBody': 'The restaurant has a wide variety of dishes, the s

{'title': 'Maravilhoso!!!!', 'reviewId': '219976718', 'restaurantId': '1015278', 'author': '81Bethania', 'reviewBody': 'The Mangai restaurant is an icon, as it knew how to mix northeastern culture and gastronomy in one place, all this combined with great comfort and organization.', 'language': 'pt', 'mobileDevice': 'False', 'userLocation': 'Natal, RN', 'publicationDate': '06/08/2014', 'visitDate': '12/2013', 'ratingValue': '5', 'itemReviewed': {'name': 'Mangai', 'address': {'streetAddress': 'Avenida Amintas Barros, 3300', 'addressLocality': '', 'addressRegion': 'Rio Grande do Norte', 'postalCode': '59075-810', 'country': 'Brasil'}, 'website': 'http://www.facebook.com/RestauranteMangai/', 'email': 'contatopb@mangai.com.br', 'phoneNumber': '+55 84 3206-3344', 'averageRating': '4.5'}, 'subjectivity': 1, 'polarity': 1}
{'title': 'Comida com qualidade e preu00e7o justo', 'reviewId': '474315990', 'restaurantId': '7828097', 'author': 'Magaliamaro', 'reviewBody': 'Delicious pasta and from Frid

<font size="3">**Open classified data**</font>

Identifies total of true positive, false positive and false negative, then we can calculate precision, recall and f1-measure.

In [9]:
import pandas as pd

dataset = pd.read_csv("data/dataset-200-classified.csv", skiprows = 2)

vp_sub = 0
fp_sub = 0
fn_sub = 0

for line, rows in dataset.iterrows():
    #VP
    if(rows['pol-check'] == 1 and rows['polarity'] == 1):
        vp_sub += 1
    #FP
    if(rows['pol-check'] == 0 and rows['polarity'] == 1):
        fp_sub +=1
    #FN
    if(rows['pol-check'] == 0 and rows['polarity'] == -1):
        fn_sub +=1

<font size="3">**Precision**</font>

Evaluate all the positive classification that the model made, and calculate how many are probably correct.

In [85]:
precision = vp_sub/(vp_sub+fp_sub)
print(str(round(precision * 100, 2))+'%')

96.77%


<font size="3">**Recall**</font>

Evaluates among all sentences with expected positive value, how many of them are correct.

In [86]:
recall = vp_sub/(vp_sub+fn_sub)
print(str(round(recall * 100, 2))+'%')

100.0%


<font size="3">**F1-measure**</font>

Evaluate the harmonic mean between precision and recall.

In [87]:
f1 = (2*(precision*recall))/(precision+recall)
print(str(round(f1 * 100, 2))+'%')

98.36%


In [83]:
measure_chart = pygal.HorizontalBar()
measure_chart.title = 'Performance evaluation for TripAdvisor dataset'
measure_chart.add('Precision', round(precision * 100, 2))
measure_chart.add('Recall', round(recall * 100, 2))
measure_chart.add('F1', round(f1 * 100, 2))
galplot(measure_chart)

<font size="4">**Research Question 3**</font>

We choose twitter as dataset for this one, because twitter has a friendly API. Then we use two words, coco bambu and habibs to find data with the restaurant subject. The dataset has a total of 200 comments

Obs: We had some problems by trying to use the API because a key is needed, and we didnt receive that key at a resonable time, so we just go to twitter and get data like a scrapper would did.

In [92]:
data_twiter = open('data/data-twiter-us.txt', 'r', encoding='utf-8')

with open('data/dataset-twitter.csv', 'w', newline='') as file:
    writer_twitter = csv.writer(file)
    writer_twitter.writerow(["comment", "subjectivity", "polarity", "sub-check", "pol-check"])

    for line_twitter in data_twiter:
        calc_twitter = TextBlob(str(line_twitter))
        writer_twitter.writerow([line_twitter, calc_twitter.sentiment.subjectivity, calc_twitter.sentiment.polarity])

    print('Done')

Done


<font size="4">**Polarity and subjectivity proportion from twitter dataset**</font>

In [103]:
import pandas as pd

dataset = pd.read_csv("data/dataset-twitter.csv", skiprows = 1)

tw_pol_positive = 0
tw_pol_negative = 0
tw_pol_neutral = 0

tw_sub_objective = 0
tw_sub_subjective = 0

for line, rows in dataset.iterrows():
    if(rows[2] > 0):
        tw_pol_positive+=1
    elif(rows[2] < 0):
        tw_pol_negative+=1
    elif(rows[2] == 0.0):
        tw_pol_neutral+=1

    if(rows[1] < 0.5):
        tw_sub_objective+=1
    else:
        tw_sub_subjective+=1

tw_pie_pol_chart = pygal.Pie(inner_radius=.4)
tw_pie_pol_chart.title = 'Polarity proportion from Twitter dataset'
tw_pie_pol_chart.add('Positive', tw_pol_positive)
tw_pie_pol_chart.add('Negative', tw_pol_negative)
tw_pie_pol_chart.add('Neutral', tw_pol_neutral)
galplot(tw_pie_pol_chart)

tw_pie_sub_chart = pygal.Pie(inner_radius=.4)
tw_pie_sub_chart.title = 'Subjectivity proportion from Twitter dataset'
tw_pie_sub_chart.add('Objective', tw_sub_objective)
tw_pie_sub_chart.add('Subjective', tw_sub_subjective)
galplot(tw_pie_sub_chart)

<font size="3">**Performance measure**</font>

In [105]:
import pandas as pd

dataset = pd.read_csv("data/dataset-twitter.csv", skiprows = 2)

tw_vp_sub = 0
tw_fp_sub = 0
tw_fn_sub = 0

for line, rows in dataset.iterrows():
    #VP
    if(rows[4] == 1 and rows[2] > 0.0):
        tw_vp_sub += 1
    #FP
    if(rows[4] == 0 and rows[2] > 0.0):
        tw_fp_sub +=1
    #FN
    if(rows[4] == 0 and rows[2] <= 0.0):
        tw_fn_sub +=1

<font size="3">**Precision**</font>

In [108]:
tw_precision = tw_vp_sub/(tw_vp_sub+tw_fp_sub)
print(str(round(tw_precision * 100, 2))+'%')

88.57%


<font size="3">**Recall**</font>

In [109]:
tw_recall = tw_vp_sub/(tw_vp_sub+tw_fn_sub)
print(str(round(tw_recall * 100, 2))+'%')

64.58%


<font size="3">**F1-measure**</font>

In [111]:
tw_f1 = (2*(tw_precision*tw_recall))/(tw_precision+tw_recall)
print(str(round(tw_f1 * 100, 2))+'%')

74.7%


In [113]:
tw_measure_chart = pygal.HorizontalBar()
tw_measure_chart.title = 'Performance evaluation for Twitter dataset'
tw_measure_chart.add('Precision', round(tw_precision * 100, 2))
tw_measure_chart.add('Recall', round(tw_recall * 100, 2))
tw_measure_chart.add('F1', round(tw_f1 * 100, 2))
galplot(tw_measure_chart)

<font size="3">**The End**</font>